Notes for Jacopo:


 <br>


Define the module path + import packages

In [ ]:
import sys
module_path = '/Users/jacoponudo/Documents/thesis/src/HWK'
sys.path.append(module_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from HWK_package.functions import *

In [ ]:
from scipy import stats
import random 
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

Set source

In [ ]:
source_data='/Users/jacoponudo/Documents/thesis/data/voat/voat_labeled_data_unified.parquet'
root='/Users/jacoponudo/Documents/thesis/'
output=root+'src/HWK/output'
output_threads=root+'src/HWK/output/temporary'

In [ ]:
dataset = pd.read_parquet(source_data)

In [ ]:
dataset['is_toxic']=dataset['toxicity_score']>0.6
user_toxic_activity=dataset.groupby(['root_submission','user'])['is_toxic'].sum().reset_index()
user_toxic_activity[user_toxic_activity['is_toxic']>10]

In [ ]:
dataset_user=dataset[dataset.user=='Mumbleberry'].copy()
dataset_user.sort_values(by='created_at', inplace=True)
dataset_user.root_submission.value_counts()
dataset_user=dataset_user[dataset_user.root_submission=='3348062'].copy()

Q1: Could we use a hawkes process to model the inter arrival time of comments?

Preparo i dati e fitto il modello 

In [ ]:
observed_data = np.array([np.datetime64(x.replace(tzinfo=None)).astype(np.int64) for x in dataset_user['created_at']])
mean_lag=np.mean(np.diff(observed_data))
ℋ_t=observed_data-min(observed_data)+mean_lag
𝛉_exp=𝛉_exp_simple=exp_mle(ℋ_t,max(ℋ_t)+mean_lag)

Faccio il QQplot del modello

In [ ]:
tsShifted = exp_hawkes_compensators(ℋ_t, 𝛉_exp)
iat = np.diff(np.insert(tsShifted, 0, 0))
qqplot(iat, dist=stats.expon, fit=True, line="45")
plt.show()

Estraggo una metric per valutare il modello

In [ ]:
slope, intercept = sm.OLS(iat, sm.add_constant(stats.expon.ppf((np.arange(1, len(iat) + 1) - 0.5) / len(iat)))).fit().params
print("Pendenza della linea di riferimento adattata:", slope)
slope, intercept = 1, 0
predicted_values = slope * np.arange(1, len(iat) + 1) + intercept

Calcolo delle distanze

In [ ]:
distances = np.abs(iat - predicted_values)

In [ ]:
print('errore modello semplice',distances.mean())

Q2: Is the fitting of the model improving using a flexibility of alpha (infetivity rate) sensible to comment's toxicity?

In [ ]:
    
# Preparo i dati e fitto il modello 
observed_data = np.array([np.datetime64(x.replace(tzinfo=None)).astype(np.int64) for x in dataset_user['created_at']])
mean_lag=np.mean(np.diff(observed_data))
ℋ_t=observed_data-min(observed_data)+mean_lag
𝒯_T=list(dataset_user.toxicity_score)
𝛉_exp=𝛉_exp_toxicity=exp_mle_toxicity(ℋ_t,𝒯_T,max(ℋ_t))

Faccio il QQplot del modello

In [ ]:
tsShifted = exp_hawkes_compensators_toxicity(ℋ_t,𝒯_T, 𝛉_exp)
iat = np.diff(np.insert(tsShifted, 0, 0))
qqplot(iat, dist=stats.expon, fit=True, line="45")
plt.show()

Estraggo una metric per valutare il modello

In [ ]:
slope, intercept = sm.OLS(iat, sm.add_constant(stats.expon.ppf((np.arange(1, len(iat) + 1) - 0.5) / len(iat)))).fit().params
print("Pendenza della linea di riferimento adattata:", slope)

In [ ]:
slope, intercept = 1, 0
predicted_values = slope * np.arange(1, len(iat) + 1) + intercept

Calcolo delle distanze

In [ ]:
distances = np.abs(iat - predicted_values)

In [ ]:
print('errore modello tossicità',distances.mean())

Q3 Posso stimare i parametri di ciascun utente studiandone la verosimiglianza congiunta su tutti i threads?<br>
--------------------------- Using α ---------------------------------------------------

In [ ]:
data_user = filter_dataset(dataset, 'Mumbleberry', min_comments=3, sample=False)
ℋ_T_list, magnitude_list,time_list = prepare_data(data_user, dataset)
θ_exp_mle_T = exponential_mle(ℋ_T_list, time_list)

In [ ]:
tsShifted = exp_hawkes_compensators(ℋ_t, θ_exp_mle_T)
iat = np.diff(np.insert(tsShifted, 0, 0))
qqplot(iat, dist=stats.expon, fit=True, line="45")
plt.show()

In [ ]:
slope, intercept = sm.OLS(iat, sm.add_constant(stats.expon.ppf((np.arange(1, len(iat) + 1) - 0.5) / len(iat)))).fit().params
print("Pendenza della linea di riferimento adattata:", slope)

In [ ]:
slope, intercept = 1, 0
predicted_values = slope * np.arange(1, len(iat) + 1) + intercept

In [ ]:
distances = np.abs(iat - predicted_values)
print('errore modello tossicità',distances.mean())

--------------------------- Using 2α ---------------------------------------------------

In [ ]:
data_user = filter_dataset(dataset, 'Mumbleberry', min_comments=3, sample=False)
ℋ_T_list, magnitude_list,time_list = prepare_data(data_user, dataset)
θ_exp_mle_T = exponential_mle_toxicity(ℋ_T_list, magnitude_list, time_list)

In [ ]:
tsShifted = exp_hawkes_compensators_toxicity(ℋ_t,𝒯_T, θ_exp_mle_T)
iat = np.diff(np.insert(tsShifted, 0, 0))
qqplot(iat, dist=stats.expon, fit=True, line="45")
plt.show()

In [ ]:
slope, intercept = sm.OLS(iat, sm.add_constant(stats.expon.ppf((np.arange(1, len(iat) + 1) - 0.5) / len(iat)))).fit().params
print("Pendenza della linea di riferimento adattata:", slope)

In [ ]:
slope, intercept = 1, 0
predicted_values = slope * np.arange(1, len(iat) + 1) + intercept

In [ ]:
distances = np.abs(iat - predicted_values)
print('errore modello tossicità',distances.mean())